# Metallic Flat Plate Magnetized in Exterior Field 
# Test Case in TU Delft Master Thesis of Lisette de Bruin 
# 2D Toy Version 

<b>Goal</b>: The goal of this notebook is to compute a 2D numerical reference solution for the magnetization vector of metallic plate $\vec{M}(\vec{x})$ placed in homogeneous external magnetic field; 

## Import Packages

In [1]:
using Ferrite, SparseArrays

## Section 1: Using Ferrite.jl 

### Ferrite.jl Howto: What are cellvalues ? 

In [2]:
?generate_grid

search: generate_grid



```
generate_grid(celltype::Cell, nel::NTuple, [left::Vec, right::Vec)
```

Return a `Grid` for a rectangle in 1, 2 or 3 dimensions. `celltype` defined the type of cells, e.g. `Triangle` or `Hexahedron`. `nel` is a tuple of the number of elements in each direction. `left` and `right` are optional endpoints of the domain. Defaults to -1 and 1 in all directions.


In [3]:
grid = generate_grid(Quadrilateral, (2, 2))

Grid{2, Quadrilateral, Float64} with 4 Quadrilateral cells and 9 nodes

In [4]:
dim = 2
ip = Lagrange{dim, RefCube, 1}()
qr = QuadratureRule{dim, RefCube}(2)
cellvalues = CellScalarValues(qr, ip)

CellScalarValues{2, Float64, RefCube} with 4 shape functions and 4 quadrature points

### Ferrite.jl Howto: What does re-init do? 

In [5]:
?reinit!

search: reinit! atreplinit parentindices CartesianIndex CartesianIndices



```
reinit!(cv::CellValues, x::Vector)
reinit!(bv::FaceValues, x::Vector, face::Int)
```

Update the `CellValues`/`FaceValues` object for a cell or face with coordinates `x`. The derivatives of the shape functions, and the new integration weights are computed.


### Ferrite.jl Howto:  How to get coords and coord_qp? 

In [6]:
function my_diff_coeff(coord_qp)
    return coord_qp[1] + coord_qp[2]
end 

my_diff_coeff (generic function with 1 method)

In [7]:
function my_get_coords(dh::DofHandler)
    # Loop over all cels
    for (cellcount, cell) in enumerate(CellIterator(dh))
        coords = getcoordinates(cell)
        # display(coords)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            coords_qp = spatial_coordinate(cellvalues, q_point, coords)
            # display(my_diff_coeff(coords_qp)) 
        end
    end
    return 0
end

my_get_coords (generic function with 1 method)

In [8]:
dh = DofHandler(grid)
add!(dh, :u, 1)
close!(dh);

In [9]:
my_coords = my_get_coords(dh)
my_coords;

## Section 2: 2D Mockup of the Magnetized Plate  

In [11]:
# define spatially varying diffusion coefficient 
function my_diff_coeff(coord_qp)
    xbound = abs(coord_qp[1])<0.3
    ybound = abs(coord_qp[2])<0.1    
    return 1+1e6*(xbound*ybound) 
end 

my_diff_coeff (generic function with 1 method)

In [12]:
# Ke: added spatially varying diffusion coefficient 
# fe: forces zero source term 
function assemble_element!(Ke::Matrix, fe::Vector, cellvalues::CellScalarValues, mycoords)
    n_basefuncs = getnbasefunctions(cellvalues)
    # Reset to 0
    fill!(Ke, 0)
    fill!(fe, 0)
    # Loop over quadrature points
    for q_point in 1:getnquadpoints(cellvalues)
        # Get the quadrature weight
        dΩ = getdetJdV(cellvalues, q_point)
        # ADDED: Get coord of quadrature point
        coords_qp = spatial_coordinate(cellvalues, q_point, mycoords)
        # ADDED: Evaluate spatially dependent diffusion coefficient in quad point 
        val_diff_coeff = my_diff_coeff(coords_qp)
        # Loop over test shape functions
        for i in 1:n_basefuncs
            δu  = shape_value(cellvalues, q_point, i)
            ∇δu = shape_gradient(cellvalues, q_point, i)
            # Add contribution to fe
            fe[i] += 0 * δu * dΩ
            # Loop over trial shape functions
            for j in 1:n_basefuncs
                ∇u = shape_gradient(cellvalues, q_point, j)
                # MODIFIED: Add contribution to Ke
                Ke[i, j] += val_diff_coeff * (∇δu ⋅ ∇u) * dΩ
            end
        end
    end
    return Ke, fe
end

assemble_element! (generic function with 1 method)

In [ ]:
coords = getcoordinates(cell)

coords_qp = spatial_coordinate(cellvalues, q_point, coords)
        
val_diff_coeff = my_diff_coeff(coords_qp)



In [13]:
function assemble_global(cellvalues::CellScalarValues, K::SparseMatrixCSC, dh::DofHandler)
    # Allocate the element stiffness matrix and element force vector
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    fe = zeros(n_basefuncs)
    # Allocate global force vector f
    f = zeros(ndofs(dh))
    # Create an assembler
    assembler = start_assemble(K, f)
    # Loop over all cels
    for cell in CellIterator(dh)
        # Added: Get coordinates from current cell 
        coords = getcoordinates(cell)
        # Reinitialize cellvalues for this cell
        reinit!(cellvalues, cell)
        # Modified - Compute element contribution
        assemble_element!(Ke, fe, cellvalues, coords)
        # Assemble Ke and fe into K and f
        assemble!(assembler, celldofs(cell), Ke, fe)
    end
    return K, f
end

assemble_global (generic function with 1 method)

In [100]:
nels  = (100, 100) # number of elements in each spatial direction
left  = Vec((-10.0, -10.0)) # start point for geometry 
right = Vec((10.0, 10.0,))    # end point for geometry
grid = generate_grid(Quadrilateral, (100, 100));

# nels  = (100, 50) # number of elements in each spatial direction
# left  = Vec((0.0, 0.0)) # start point for geometry 
# right = Vec((2.0, 1.0,))    # end point for geometry
# grid = generate_grid(Quadrilateral, (50, 50),left, right);
#grid 

In [101]:
dim = 2
ip = Lagrange{dim, RefCube, 1}()
qr = QuadratureRule{dim, RefCube}(2)
cellvalues = CellScalarValues(qr, ip);

In [102]:
dh = DofHandler(grid)
add!(dh, :u, 1)
close!(dh);

In [103]:
K = create_sparsity_pattern(dh);

In [104]:
ch = ConstraintHandler(dh);

In [105]:
∂Ω = union(
    getfaceset(grid, "left"),
    getfaceset(grid, "right"),
    getfaceset(grid, "top"),
    getfaceset(grid, "bottom"),
);

# ∂Ω = union(
#    getfaceset(grid, "right"),
#    getfaceset(grid, "top"),
# );

In [106]:
dbc = Dirichlet(:u, ∂Ω, (x, t) -> x[1])
add!(ch, dbc);

In [107]:
close!(ch)

ConstraintHandler:
  BCs:
    Field: u, Components: [1]

In [108]:
K, f = assemble_global(cellvalues, K, dh);

In [109]:
apply!(K, f, ch)
u = K \ f;

### Computing Fluxes

In [118]:
function compute_hfield(cellvalues::CellScalarValues{dim,T}, dh::DofHandler, a) where {dim,T}

    n = getnbasefunctions(cellvalues)
    cell_dofs = zeros(Int, n)
    nqp = getnquadpoints(cellvalues)

    # Allocate storage for the fluxes to store
    q = [Vec{2,T}[] for _ in 1:getncells(dh.grid)]

    for (cell_num, cell) in enumerate(CellIterator(dh))
        q_cell = q[cell_num]
        celldofs!(cell_dofs, dh, cell_num)
        aᵉ = a[cell_dofs]
        reinit!(cellvalues, cell)

        for q_point in 1:nqp
            q_qp = - function_gradient(cellvalues, q_point, aᵉ)
            push!(q_cell, q_qp)
        end
    end
    return q
end

function compute_bfield(cellvalues::CellScalarValues{dim,T}, dh::DofHandler, a) where {dim,T}

    n = getnbasefunctions(cellvalues)
    cell_dofs = zeros(Int, n)
    nqp = getnquadpoints(cellvalues)

    # Allocate storage for the fluxes to store
    q = [Vec{2,T}[] for _ in 1:getncells(dh.grid)]

    for (cell_num, cell) in enumerate(CellIterator(dh))
        q_cell = q[cell_num]
        celldofs!(cell_dofs, dh, cell_num)
        aᵉ = a[cell_dofs]
        reinit!(cellvalues, cell)

        for q_point in 1:nqp
            q_qp = - function_gradient(cellvalues, q_point, aᵉ)
            push!(q_cell, q_qp)
        end
    end
    return q
end

compute_hfield (generic function with 1 method)

In [119]:
q_gp = compute_hfield(cellvalues, dh, u);
#q_gp

In [112]:
projector = L2Projector(ip, grid);
#projector 

In [113]:
#?project

In [114]:
q_projected = project(projector, q_gp, qr; project_to_nodes=false);
# q_projected

### Exporting to VTK 

In [115]:
vtk_grid("magnetic_plate", dh) do vtk
    vtk_point_data(vtk, dh, u)
    vtk_point_data(vtk, projector, q_projected, "q")
end

1-element Vector{String}:
 "magnetic_plate.vtu"